In [ ]:
import pandas as pd
import torch
import sys
import os
import pandas as pd
from scipy.io import loadmat
from tqdm import tqdm
from glob import glob
import numpy as np
from collections import defaultdict
from glob import glob
from scipy.signal import spectrogram
sys.path.append('../src')

/data/home/jkataok1/mlenv/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
signal_size = 2048
root = "/data/home/jkataok1/DA_DFD/data/raw/PU"
#bearing_codes = ["K001", "KI14", "KI18", "KA04", "KA16", "KB23", "KB24"]
working_condtions = ["N15_M07_F10","N09_M07_F10","N15_M01_F10","N15_M07_F04"]

healthy_code = ["K001", "K002", "K003", "K004", "K005"]
outer_rind_code = ["KA04", "KA15", "KA16", "KA22", "KA30"]
inner_ring_code = ["KI01", "KI14", "KI16", "KI18", "KI21"]

bearing_codes = healthy_code + outer_rind_code + inner_ring_code

def data_load(filename,name):
    '''
    This function is mainly used to generate test data and training data.
    filename:Data location
    '''
    fl = loadmat(filename)[name]
    fl = fl[0][0][2][0][6][2]  #Take out the data
    fl = fl.reshape(-1,1)
    data=[] 
    start,end=0, signal_size
    while end<=fl.shape[0]:
        #data.append(fl[start:end])
        signal = fl[start:end].reshape(-1)
        f, t, sxx = spectrogram(signal, fs=2048, nperseg=128)
        data.append(sxx)
        start += signal_size
        end += signal_size

    data = np.stack(data, axis=0)

    return data


def load_pu(root, bearing_codes, working_condtion):
    data = []
    label = []
    for i in range(len(bearing_codes)):
        file_names = glob(os.path.join(root,  bearing_codes[i], working_condtion + '*'))
        for f in file_names:
            f_name = os.path.basename(f).split('.')[0]

            f_data = data_load(f, f_name)
            data.append(f_data)
            if bearing_codes[i] in healthy_code:
                label.append(np.repeat(0, len(f_data)))
            elif bearing_codes[i] in outer_rind_code:
                label.append(np.repeat(1, len(f_data)))
            elif bearing_codes[i] in inner_ring_code:
                label.append(np.repeat(2, len(f_data)))

    data = np.expand_dims(np.concatenate(data), axis=1)
    data = (data - data.mean(axis=(0, -2, -1), keepdims=True)) / (data.std(axis=(0, -2, -1), keepdims=True)+1e-8)
    label = np.expand_dims(np.concatenate(label), axis=-1)

    return data, label

save_path = "/data/home/jkataok1/DA_DFD/data/processed/PU"
for i in range(len(working_condtions)):
    data, label = load_pu(root, bearing_codes, working_condtions[i])
    np.savez(os.path.join(save_path, f'{i}_spectrogram.npz'), x=data, y=label)

    



In [11]:
x = np.load("/data/home/jkataok1/DA_DFD/data/processed/PU/0_spectrogram.npz")["x"]
y = np.load("/data/home/jkataok1/DA_DFD/data/processed/PU/0_spectrogram.npz")["y"]